In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
# loding iris dataset
iris = datasets.load_iris()
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [3]:
# calculate entropy of the dataset
# entropy = -sigma(pi log(pi))
# pi = count[i] / total
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts = True)
    entropy = np.sum([(-counts[i] / np.sum(counts)) * np.log2(counts[i]) / np.sum(counts) for i in range(len(elements))])
    return entropy

In [4]:
# calculating information gain
"""
    data -> dataset
    split_attribute_name -> the name of feature for which information gain should be calculated
    target_name -> the name of the target feature
"""
def information_gain(data, split_attribute_name, target_name = "class"):
    # calculate entropy of the dataset
    total_entropy = entropy(data[target_name])
    
    # calculate the values and counts for split attribute
    values, counts = np.unique(data[split_attribute_name], return_counts = True)
    
    # calculate weighted entropy
    weighted_entropy = np.sum([(counts[i] / np.sum(counts)) * entropy(data.where(data[split_attribute_name] == values[i]).dropna()[target_name]) for i in range(len(values))])
    
    # calculate the information gain
    information_gain = total_entropy - weighted_entropy
    return information_gain

In [5]:
def decision_tree(data, original_data, features, target_attribute_name = "target", parent_node_class = None):
    """
        data -> data on which decision tree should run, for first iteration it is complete dataset
        original_data -> orginal dataset
        features -> features in the dataset
        target_attribute_names = name of data attribute
        parent_node_class = if no features left to split on we reach to parent
    """
    # if only one feature is left
    if(len(np.unique(data[target_attribute_name]))) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    # check for empty dataset
    elif len(data) == 0:
        return np.unique(original_data[target_attribute_name])[np.argmax(np.unique(original_data[target_attribute_name], return_counts = True)[1])]
    
    # if feature space is empty, return feature value of direct parent
    elif len(features) == 0:
        return parent_node_class
    
    # else grow the tree
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name], return_counts= True)[1])]
        
        # select the feature which best splits the data
        # find the information gain for all features
        item_values = [information_gain(data, feature, target_attribute_name) for feature in features]
        
        # choose feature with max information gain
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
#         print('best feature', best_feature)
        # create the tree with root named after the best feature name
        tree = {best_feature:{}}
        
        # remove the feature on which the split has been made
        features = [i for i in features if i != best_feature]
        
        # grow the branches of the tree for each possible value of the root node
        for value in np.unique(data[best_feature]):
            # split the dataset along the value of the feature with largest information gain
            sub_data = data.where(data[best_feature] == value).dropna()
            
            # make recursive calls
            subtree = decision_tree(sub_data, original_data, features, target_attribute_name, parent_node_class)
            
            # add subtree under the root of the tree
            tree[best_feature][value] = subtree
        return tree

In [6]:
# writing the predict function
def predict(query, tree, default = 1):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]]
            except:
                return default
            result = tree[key][query[key]]
            if isinstance(result, dict):
                return predict(query, result) # recursive call
            else:
                return result

In [7]:
# testing our tree
def test(data, tree):
    queries = data.iloc[:, :-1].to_dict('records')
    predict_and_test = pd.DataFrame(columns=['Predicted', 'True_values'])
    
    # calculate the predicted accuracy
    for i in range(len(data)):
        predict_and_test.loc[i, "Predicted"] = predict(queries[i], tree, 1.0)
        predict_and_test.loc[i, "True_values"] = data['target'].iloc[i]
    # printing the accuracy
    # accuracy = correct_predicted / total_predcited * 100
    # print(data['target'])
    # print(predict_and_test)
    print("The prediction accuracy is: ", np.sum((predict_and_test["Predicted"] == predict_and_test["True_values"]) / len(data)) * 100, '%')

In [8]:
# creating the dataset
df = pd.DataFrame()

In [9]:
df['sepal length (cm)'] = [iris.data[i][0] for i in range(len(iris.data))]
df['sepal width (cm)'] = [iris.data[i][1] for i in range(len(iris.data))]
df['petal length (cm)'] = [iris.data[i][2] for i in range(len(iris.data))]
df['petal width (cm)'] = [iris.data[i][3] for i in range(len(iris.data))]
df['target'] = [iris.target[i] for i in range(len(iris.target))]

In [10]:
# shuffling the data
df = shuffle(df).reset_index(drop=True)
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [11]:
# let us split the data in ratio 75:25
training_data = df.iloc[:112]
testing_data = df.iloc[112:]
training_data.shape, testing_data.shape

((112, 5), (38, 5))

In [12]:
# call for functions
# train the tree
tree = decision_tree(training_data, training_data, training_data.columns[:-1])
print(tree)

{'petal length (cm)': {1.1: 0.0, 1.2: 0.0, 1.3: 0.0, 1.4: 0.0, 1.5: 0.0, 1.6: 0.0, 1.7: 0.0, 1.9: 0.0, 3.3: 1.0, 3.5: 1.0, 3.6: 1.0, 3.7: 1.0, 3.8: 1.0, 3.9: 1.0, 4.0: 1.0, 4.1: 1.0, 4.2: 1.0, 4.4: 1.0, 4.5: {'sepal width (cm)': {2.5: 2.0, 2.8: 1.0, 3.0: 1.0, 3.4: 1.0}}, 4.6: 1.0, 4.7: 1.0, 4.8: {'sepal length (cm)': {5.9: 1.0, 6.2: 2.0, 6.8: 1.0}}, 4.9: {'petal width (cm)': {1.5: 1.0, 1.8: 2.0, 2.0: 2.0}}, 5.0: {'sepal length (cm)': {5.7: 2.0, 6.7: 1.0}}, 5.1: {'sepal length (cm)': {5.8: 2.0, 6.0: 1.0, 6.3: 2.0, 6.5: 2.0, 6.9: 2.0}}, 5.2: 2.0, 5.3: 2.0, 5.4: 2.0, 5.5: 2.0, 5.6: 2.0, 5.7: 2.0, 5.8: 2.0, 5.9: 2.0, 6.0: 2.0, 6.1: 2.0, 6.3: 2.0, 6.4: 2.0, 6.6: 2.0, 6.7: 2.0, 6.9: 2.0}}


In [13]:
test(testing_data, tree)

The prediction accuracy is:  0.0 %
